In [1]:
import catboost
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('dataset/prep.csv')

# Preprocessing

### Change data to use CatBoost

In [3]:
def replace_from_dict(dct, column):
    return list(map(lambda x: dct.get(x) if x in dct else x, data[column]))

#### Cuzov

In [4]:
dict_cuzov = {'седан':'sedan', 
               'хэтчбек':'hatchback',
               'внедорожник':'SUV',
               'универсал':'universal',
               'минивэн':'minivan',
               'лифтбек':'liftback',
               'микроавтобус':'minibus',
               'купе':'coupe',
               'легковой':'passenger',
               'пикап':'pickup',
               'кабриолет':'cabriolet',
               'другой':'other',
               'лимузин':'limousine',
               'родстер':'roadster'               
}

In [5]:
data.cuzov = replace_from_dict(dict_cuzov, 'cuzov')

#### Fuel

In [6]:
dict_fuel = {'бензин':'petrol', 
             'дизель':'diesel'               
}

In [7]:
data.fuel = replace_from_dict(dict_fuel, 'fuel')

#### Brand

In [8]:
dict_brand = {'ГАЗ':'GAZ', 
             'УАЗ':'UAZ',
             'Москвич':'Moskvich', 
             'ЗАЗ':'ZAZ', 
             'Эксклюзив':'Exclusive',
             'ЛуАЗ':'LuAZ'
}

In [9]:
data.brand = replace_from_dict(dict_brand, 'brand')

#### Region

In [10]:
dict_region = {'Минск':'Minsk', 
               'Брестская обл.':'Brest reg.',
               'Гродненская обл.':'Grodno reg.', 
               'Гомельская обл.':'Gomel reg.', 
               'Могилевская обл.':'Mogilev reg.',
               'Витебская обл.':'Vitebsk reg.',
               'Минская обл.':'Minsk reg.'
}

In [11]:
data.region = replace_from_dict(dict_region, 'region')

#### Model - drop

In [12]:
data.drop('model', axis=1, inplace=True)

#### Color

In [13]:
data.color.unique()

array(['черный', 'серебристый', 'зеленый', 'фиолетовый', 'синий',
       'бордовый', 'серый', 'белый', 'другой', 'красный', 'коричневый',
       'оранжевый', 'желтый'], dtype=object)

In [14]:
russian_colors = ['черный', 'серебристый', 'зеленый', 'фиолетовый', 'синий',
                  'бордовый', 'серый', 'белый', 'другой', 'красный', 
                  'коричневый', 'оранжевый', 'желтый']
english_colors = ['black', 'silver', 'green', 'purple', 'blue',
                  'burgundy', 'gray', 'white', 'other', 'red',
                  'brown', 'orange', 'yellow']
dict_colors = dict(zip(russian_colors, english_colors))

In [15]:
data.color = replace_from_dict(dict_colors, 'color')

#### Drive Unit

In [16]:
dict_drive_unit = {'Передний':'front-wheel', 
                   'Постоянный полный':'all-wheel', 
                   'Задний':'rear', 
                   'Подключаемый полный':'four-wheel'               
}

In [17]:
data['drive-unit'] = replace_from_dict(dict_drive_unit, 'drive-unit')

In [18]:
data['drive-unit'].fillna('Unknown', inplace=True)

#### State

In [19]:
dict_state = {'с пробегом':'with mileage', 
              'с повреждениями':'with damage', 
              'на запчасти':'for parts'               
}

In [20]:
data.state = replace_from_dict(dict_state, 'state')

#### Transmission

In [21]:
dict_transmission = {'Автомат':'automatic', 
                     'Механика':'mechanical'               
}

In [22]:
data.transmission = replace_from_dict(dict_transmission, 'transmission')

#### Drop views

In [23]:
data.drop(['all_views', 'today_views', 'up', 'days_ago'], axis=1, inplace=True)

In [24]:
data.drop('modified', axis=1, inplace=True)

In [25]:
data.cost = data.cost.str.replace(' ', '').astype('int32')

# Train Test Split

In [28]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('cost', axis=1), 
                                                    data.cost, random_state=42,
                                                    test_size=0.2)

# Remove Outliers